In [1]:
%run ./libs.ipynb
%run ./initialize.ipynb
%run ./signals_sec312.ipynb

from ipywidgets import TwoByTwoLayout
from IPython.display import clear_output

Noversampl = 1 
AutoPhase = True
SNR = True
IQ = True

ph0 = 0.4

In [ ]:
def getSig(_):
    Order = order_dropdown.value + 1
    FiltType = FiltType_dropdown.value
    FiltRange = FiltRange_slider.value
    ndepth = ndepth_text.value
    AVG = avg_text.value
    
    nfilt = int((FiltRange[1]-FiltRange[0])/0.1)
               
    global snr
    snr = np.zeros([nfilt, ndepth])
    
    filtrange = np.linspace(FiltRange[0], FiltRange[1], nfilt)
    f = np.linspace(0, 1, ndepth)
    
    for nn  in range(0,nfilt):
        print("Calculating: ", int(nn/nfilt*100), "%")
        FILTFACT = filtrange[nn]
                
        for avg in range(0, AVG):
            snr0 = signals_sec312(Noversampl, oof, ofcont, FILT, Order, FiltType, FILTFACT, AutoPhase, ph0, IQ, noise, SNR, ndepth)

            snr[nn, :] = snr[nn, :] + snr0
                
    getPlot(10*np.log10(snr/AVG),filtrange,f)   
                  
def getPlot(y,filtrange,f):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    plot1 = plt.imshow(y, extent=[f[0],f[-1],filtrange[-1],filtrange[0]], cmap=plt.cm.viridis)
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.ylabel('Cutoff, nfo', fontsize = 15)
    plt.xlabel('Baseband', fontsize = 15)
    plt.show()

In [8]:
# order    
order_dropdown = widgets.Dropdown(options = [('2',1),('3',2),('4',3),('5',4)],
                                  value = 4,
                                  description = 'Order:',
                                  layout=Layout(width='180px'))

# filter type
FiltType_dropdown = widgets.Dropdown(options = [('Butter', 1), ('Cheby', 2)],
                                     value = 1,
                                     description = 'Type:',
                                     layout=Layout(width='180px'))

# corner frequency range
FiltRange_slider = widgets.FloatRangeSlider(value = [1, 1.8],
                                            min=0.7, max=3, step=0.1,
                                            description = 'LPF cutoff',
                                            readout_format='.1f')
# number of depths
ndepth_text = widgets.IntText(value = 10, description = '# depths',  layout=Layout(width='180px'))

# number of averages
avg_text = widgets.IntText(value = 10, description = 'Avg',  layout=Layout(width='180px'))

# run calculation
Run_button = widgets.Button(description='Run', button_style = 'danger', layout=Layout(width='92px', margin='1px 90px')) 

In [ ]:
Run_button.on_click(getSig)